In [1]:
import pandas as pd
from haversine import haversine
import json
import numpy as np
import datetime
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/center_of_rentrak_zip.json"))

In [2]:
store_level_zips=pd.read_csv("/home/jian/Projects/Big_Lots/New_TA/zips_in_new_ta/sales_by_zip (Store level).csv",dtype=str)
store_level_zips=store_level_zips[store_level_zips['revenue_flag'].isin(['P', 'S'])].drop_duplicates()
store_level_zips_excluded=store_level_zips[store_level_zips['location_id'].isin(['5215','1840'])].drop_duplicates()
store_level_zips_other=store_level_zips[~store_level_zips['location_id'].isin(['5215','1840'])].drop_duplicates()
store_level_zips_other_of_exclusion=store_level_zips_other[store_level_zips_other['zip'].isin(store_level_zips_excluded['zip'])].drop_duplicates()

In [3]:
store_sales=pd.read_csv("/home/jian/BiglotsCode/outputs/combined_sales_long_2018-10-06.csv",dtype=str)
store_sales['sales']=store_sales['sales'].astype(float)
store_sales=store_sales[store_sales['week_end_date']=="2018-10-06"]
store_sales=store_sales[store_sales['sales']>0]
open_store_list=store_sales['location_id'].unique().tolist()

In [4]:
store_level_zips_excluded=store_level_zips_excluded[['location_id','revenue_flag','zip']]
store_level_zips_excluded.columns=[x+"_exclusion" for x in store_level_zips_excluded.columns.tolist()]


In [5]:
store_level_zips_other_of_exclusion=store_level_zips_other_of_exclusion[['location_id','revenue_flag','zip']]
store_level_zips_other_of_exclusion.columns=[x+"_other_inflected" for x in store_level_zips_other_of_exclusion.columns.tolist()]
store_level_zips_other_of_exclusion=store_level_zips_other_of_exclusion.rename(columns={"zip_other_inflected":"zip_exclusion"})
store_level_zips_other_of_exclusion=store_level_zips_other_of_exclusion[store_level_zips_other_of_exclusion['location_id_other_inflected'].isin(open_store_list)]

In [6]:
store_level_zips_other_of_exclusion['labels_for_other_stores']="{"+store_level_zips_other_of_exclusion['location_id_other_inflected']+": "+store_level_zips_other_of_exclusion['revenue_flag_other_inflected']+"}"
store_level_zips_other_of_exclusion=store_level_zips_other_of_exclusion.groupby(['zip_exclusion'])['labels_for_other_stores'].apply(list).to_frame().reset_index()

output=pd.merge(store_level_zips_excluded,store_level_zips_other_of_exclusion,on="zip_exclusion",how="outer")
store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20181001-135417-132.txt",sep="|",dtype=str)
store_sales[store_sales['location_id'].apply(lambda x: x in store_list['location_id'].tolist())==False]
# Checked from the list from Dom and it's in WA, far away from the 2 exclusions

,DMA,location_id,sales,traffics,transactions,week_end_date,week_indicator
215146,NaN,4681,88457.47,5444,2477.0,2018-10-06,21


In [7]:
store_list=store_list[store_list['location_id'].isin(open_store_list)][['location_id','latitude_meas','longitude_meas']]
store_list['latitude_meas']=store_list['latitude_meas'].astype(float)
store_list['longitude_meas']=store_list['longitude_meas'].astype(float)
store_list=store_list[~store_list['location_id'].isin(['5215','1840'])]
store_list=store_list.reset_index()
del store_list['index']
store_list_lat_dict=store_list.set_index(['location_id']).to_dict()['latitude_meas']
store_list_lng_dict=store_list.set_index(['location_id']).to_dict()['longitude_meas']

In [8]:
output=output.reset_index()
del output['index']
output['other_store_in_10_miles']=np.nan
for i in range(len(output)):
    zip_exclusion=output['zip_exclusion'][i]
    zip_dict_with_sotres={}
    
    if zip_exclusion in list(zip_centers.keys()):
        for store in store_list['location_id'].tolist():

            store_center=[store_list_lat_dict[store],store_list_lng_dict[store]]
            dist=haversine(store_center,zip_centers[zip_exclusion],miles=True)

            if dist<=10:
                zip_dict_with_sotres.update({store:dist})
        output['other_store_in_10_miles'][i]=str(zip_dict_with_sotres)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/usr/local/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [13]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/New_TA/Store_Level_Zip_Selection/excluding_zips_for_stores/BL_zip_exclusions_for_2_stores_5215_1840_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
output=output.sort_values(['location_id_exclusion','revenue_flag_exclusion'])
output.to_excel(writer,"zip_list_for_2_stores",index=False)
writer.save()

In [12]:
output.head(2)

,location_id_exclusion,revenue_flag_exclusion,zip_exclusion,labels_for_other_stores,other_store_in_10_miles
0,1840,P,32446,"[{5169: S}, {5239: S}]",{}
2,1840,P,32460,NaN,{}
